In [1]:
import pandas as pd
import numpy as np
import glob as glob

In [8]:
def getMedMentions(condition):
    data_file = 'Final_processed_reviews/{:s}_processed.csv'.format(condition)
    df = pd.read_csv(data_file, sep='$', index_col=0)

    meds_file = 'UniqueMedications/Medications_unique_{:s}.csv'.format(condition)
    medications = [med.strip() for med in pd.read_csv(meds_file, sep='$')['Medication']]

    columns_to_process = [col for col in df.columns if col not in (medications+['Effectiveness',
                                                                                'Satisfaction',
                                                                                'Positive polarity',
                                                                                'Negative polarity',
                                                                                'Full Review',
                                                                                'Age',
                                                                                'Gender',
                                                                                'Length of treatment'])]

    # Dropping all columns that aren't side effects or where a side effect shows up in fewer than    
    # 0.5% of the reviews ---> WHAT ABOUT MEDS THAT HAVE A NUMBER OF REVIEWS ON THIS ORDER?          
    # ---> DROPPING MEDS WITH LESS THAN 50 reviews, don't have more than 10,000 total                
    list_col = columns_to_process.copy()
    for col in list_col:
        if df[col].sum() <= 0.005*df.shape[0]:
            columns_to_process.remove(col)

    return columns_to_process


all_SEs = []
for condition in ['ADHD', 'Anxiety', 'Bipolar-Disorder', 'Depression', 'Schizophrenia']:
    all_SEs += getTop995Perc(condition)

uniq_SEs = np.unique(all_SEs)


In [9]:
len(uniq_SEs)

354

In [24]:
clusters = ['Doesnt fit',
            'Weight changes', 
            'Mood and behavioral changes', 
            'Vision changes',
            'Headaches and',
            'Body aches and pain',
            'Memory and concentration issues',
            'Menstrual changes',
            'Sleep issues and drowsiness',
            'Balance, coordination, and muscle control',
            'Dizziness and fainting',
            'Stomach issues',
            'Intestinal issues',
            'Skin issues',
            'Dry mouth and changes in taste',
            'Blood sugar changes',
            'Hair loss and abnormal growth',
            'Changes in libido and sexual performance',
            'Changes in energy',
            'Sweating and temperature control issues',
            'Eye itching or sensitivity changes',
            'Blood pressure and heart rate changes',
            'Changes in appetite',
            'Urinary changes',
            'Kidney issues',
            'Hearing issues',
            'Respiratory issues and coughing',
            'Salivary issues',
            'Breast growth and swelling (all genders)',
            'Dental issues'
            ]

# for i, c in enumerate(clusters):
#     print(i,'\t',c,'\n')

In [30]:
clustered2 = {}
for cluster in clusters: 
    clustered2[cluster] = []
    
test_set = np.unique(np.array(clustered['Doesnt fit']))
for SE in test_set:
    category = input(SE)
    clustered2[clusters[int(category)]].append(SE)

bilirubin, level, elevate, blood, increase24
bipolar, affective, depressive, manic, disorder0
black, tongue27
buzz, nonspecific, hear, symptom, sensation25
cause0
certain, interaction, medical, take, drug0
certain, interaction, take, medical, change0
clinical, disease, granulomatous, medication, pituitary24
clinical, medication, kidney, granulomatous, pituitary24
clinical, medication, pituitary, granulomatous, kidney24
clot, blood0
continue, ring, buzz, unexplained, noise, ear25
cough, blood26
defibrillation, ventricular, asystolic, usually, heart0
dehydration, medication, diabetes, reaction0
diabetes0
difficulty, breathe26
ear, congestion25
effect, sensitivity, nasal, adverse, vomit11
feel0
general, feel, discomfort, illness0
great, diastolic, kidney, document, chronic24
hear, buzz, nonspecific, symptom, sensation25
hear, los25
illness, discomfort, general, feel, characterize0
illness, discomfort, general, feel, disorder0
illness, general, discomfort, feel, characterize0
illness, gene

In [31]:
clustered.pop('Doesnt fit')

['bilirubin, level, elevate, blood, increase',
 'bipolar, affective, depressive, manic, disorder',
 'black, tongue',
 'buzz, nonspecific, hear, symptom, sensation',
 'cause',
 'certain, interaction, medical, take, drug',
 'certain, interaction, take, medical, change',
 'clinical, disease, granulomatous, medication, pituitary',
 'clinical, medication, kidney, granulomatous, pituitary',
 'clinical, medication, pituitary, granulomatous, kidney',
 'clot, blood',
 'continue, ring, buzz, unexplained, noise, ear',
 'cough, blood',
 'defibrillation, ventricular, asystolic, usually, heart',
 'dehydration, medication, diabetes, reaction',
 'diabetes',
 'difficulty, breathe',
 'ear, congestion',
 'effect, sensitivity, nasal, adverse, vomit',
 'feel',
 'general, feel, discomfort, illness',
 'great, diastolic, kidney, document, chronic',
 'hear, buzz, nonspecific, symptom, sensation',
 'hear, los',
 'illness, discomfort, general, feel, characterize',
 'illness, discomfort, general, feel, disorder',

In [33]:
for key in clustered2:
    if key not in clustered:
        clustered[key] = clustered2[key].copy()
    elif len(clustered2[key]) >= len(clustered[key]):
        clustered[key] = clustered2[key].copy()
        
for key in clustered:
    clustered[key] = list(np.unique(np.array(clustered[key])))

In [44]:
np.array(clustered['Weight changes'], dtype=str)

array(['gain, weight, body, increase', 'increase, weight',
       'lose, reduction, reduce, weight, body',
       'lose, reduction, weight, body, reduce',
       'lose, reduction, weight, reduce, body', 'rapid, weight, gain',
       'reduction, lose, weight, reduce, body',
       'reduction, lose, weight, reduce, loss',
       'unusual, increase, decrease, weight',
       'unusual, weight, gain, loss', 'weight, gain', 'weight, los',
       'weight, loss'], dtype='<U37')

In [51]:
# Making a side effect clustering dataframe
cluster_set = []
for key in clustered:
    cluster = {}
    cluster['Index'] = key
    for item in clustered[key]:
        cluster[item] = 1
    cluster_set.append(cluster)
    
df = pd.DataFrame(cluster_set).fillna(0).set_index('Index').drop(index=['Doesnt fit'])

In [53]:
df.to_csv('cluster_file.csv', sep='$')